## ALLAIN Louis - TP2 - Le problème du sac

## 1. Approche force brute

### Initialisation

In [0]:
import random 
import itertools
import time

# renvoi un ensemble d'items sous la forme [ [1, 2], [4, 2] ] #gain, poids
# n : nombre d'items
def genere_items(n):
    items = []
    for i in range(0, n):
        items.append([random.randint(1, 10), random.randint(1, 10)])
    
    return items

n = 5
M = 5*n
items = genere_items(n)
print(f"Capacite : {M}")
print(f"Nb items : {n}")
print(f"Items : {items}")

Capacite : 25
Nb items : 5
Items : [[5, 3], [5, 9], [6, 1], [10, 4], [9, 3]]


### 1.1. Sans répétition

In [0]:
# calcul toutes les combinaisons possibles (sans répétition), les évalue
# et renvoie le gain optimal
def SOLVE_BAG_BRUTE_FORCE(items, M, n):
    possible_combinations = []
    for i in range(1, n+1):
        possible_combinations.append(list(itertools.combinations(items, i)))
    
    weight_saved = 0
    weight_current = 0
    for comb_list_by_size in possible_combinations:
        for comb in comb_list_by_size:
            for elem in comb:
                weight_current = weight_current + elem[1]
            
            if(weight_current > weight_saved and weight_current <= M):
                weight_saved = weight_current
                
    return weight_saved

tmp_debut = time.perf_counter()
best_w = SOLVE_BAG_BRUTE_FORCE(items, M, n)
tmp_fin = time.perf_counter()
print(f"Meilleur poids : {best_w}")
print(f"Temps d'exécution : {tmp_fin-tmp_debut}")

Meilleur poids : 20
Temps d'exécution : 0.00021639999999933934


### 1.2. Avec répétitions

In [0]:
# détermine le nombre de fois max d'utilisation par item
items_rep = []
for elem in items:
    f = M//elem[1]
    for i in range(f): 
        items_rep.append(elem)

tmp_debut = time.perf_counter()
best_w = SOLVE_BAG_BRUTE_FORCE(items_rep, M, n)
tmp_fin = time.perf_counter()
print(f"Meilleur poids : {best_w}")
print(f"Temps d'exécution : {tmp_fin-tmp_debut}")

Meilleur poids : 24
Temps d'exécution : 2.5278971000000183


## 2. Approche gloutonne

### Initialisation

In [0]:
# Trie un tableau d'items dans l'ordre décroissant selon leur ration gain/masse
# Renvoie un nouveau tableau où on a ajouté à chaque élément son ration correspondant
# Le retour est de la forme [ [1, 2, 0,5], [4, 2, 2]  ] #gain, poids, ratio
def trieItemsParRatio(items):
    
    ret = items.copy()
    
    # Utilisé pour comparer selon le ratio
    def takeThird(elem):
        return elem[2]
    
    for elem in ret:
        elem.append(elem[0]/elem[1])
        
    ret.sort(key=takeThird, reverse=True)
    return ret

### 2.1. Sans répétition

In [0]:
def SOLVE_BAG_GLOUTON(items, M, n):
    
    items_trie = trieItemsParRatio(items)

    current_weight = 0
    best_weight = 0

    for elem in items_trie:
        current_weight = current_weight + elem[1]
        if(current_weight > M):
            break
        best_weight = best_weight + elem[1]
    return best_weight

tmp_debut = time.perf_counter()
best_w = SOLVE_BAG_GLOUTON(items, M, n)
tmp_fin = time.perf_counter()
print(f"Meilleur poids : {best_w}")
print(f"Temps d'exécution : {tmp_fin-tmp_debut}")

Meilleur poids : 20
Temps d'exécution : 9.499999998752173e-05


### 2.2. Avec répétition

In [0]:
tmp_debut = time.perf_counter()
best_w = SOLVE_BAG_GLOUTON(items_rep, M, n)
tmp_fin = time.perf_counter()
print(f"Meilleur poids : {best_w}")
print(f"Temps d'exécution : {tmp_fin-tmp_debut}")

Meilleur poids : 25
Temps d'exécution : 0.00026410000003807


## 3. Programmation dynamique

In [0]:
def SOLVE_BAG_DYNAMIQUE(W, wt, val, n): 
    K = [[0 for x in range(W+1)] for x in range(n+1)] 
    for i in range(n+1): 
        for w in range(W+1): 
            if i==0 or w==0: 
                K[i][w] = 0
            elif wt[i-1] <= w: 
                K[i][w] = max(val[i-1] + K[i-1][w-wt[i-1]],  K[i-1][w]) 
            else: 
                K[i][w] = K[i-1][w] 
    return K[n][W]